# Spam Filter for Look2Social

## ~03/10

* Finished Collecting Data
* Communicated with Look2Social team: Figured out the goal of the program
* Communicated with instructors: Adjusted my approach to the project

## 03/14 ~ : Data Cleaning

### 03/14 - Overview of data
(screenshot of original data)
* 20,000 rows of data - not labeled for spam
* Six criteria for spam: 

    1) Marketing Focus (by social media marketing group)
    
    2) Bot Generated
    
    3) Known Spammer
    
    4) Corporate Posted
    
    5) Own Post
    
    6) Hijacking (just use tag, different content)


Bot Generated - can be labeled by either pattern of post ("just closed a deal...") and url

Corporate Posted - User input account name

Known spammer, Own Post - User input account name

Hijacking, Marketing - Need some eyeballing.

In [1]:
import pandas as pd
import numpy as np

### Suggestion from Miles

(pic of Miles)

"Use Machine Learning form early phase! Use it for labeling!" 

### Approach
1. create 7 columns for spam - one column per each criteria 
    one for final spam column that's default value is 0 but become 1 based on user declaration of spam

2. label bot generated and corporate generated first by their pattern

3. seperate out the rows with no label. (~7000 rows)

4. label first dozen~hundred for hijacking and marketing

5. use machine learning (Naive Bayes / Logistic Regression / Random Forest) to label, set up threshold high 

6. eyeball rest of 'em

### 03/17 - Steps before labeling data
Created new columns and index

In [2]:
df = pd.read_excel('data/data_prepping_zeros_index.xlsx')

In [3]:
df.head()

,index,text,retweet_count,favorite_count,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,...,place_type,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url
0,0,just closed a deal in 27 hours using #AdobeSig...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Fe0YfarG31
1,1,just closed a deal in 2 days using #AdobeSign ...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/TTlCvaDE0V
2,2,just closed a deal in 2 hours using #AdobeSign...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/okmHDeM9bZ
3,3,just closed a deal in 26 hours using #AdobeSig...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/AAhUGDLZxH
4,4,just closed a deal in 6 days using #AdobeSign ...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Oi57q6jdXP


1. Corporate post

Criteria for corporate = client(DocuSign) and its competitors (OneSpan, SignNow, Adobe Sign)

Explored 'screen_name' column and came up with following list of corporate account name:

In [4]:
adobe = ['Acrobat','Acrobat_GU','adobe_mabhatia','adobe1234567','AdobeCare','AdobeDocCloud','AdobeExpCloud','AdobeGov','adobemax','AdobeNews','AdobePartner','adobesignstx','AdobeStarProps','AdobeUK']
onespan = ['atOneSpan','OneSpan','OneSpanSign']
docusign = ['DocuSign','DocuSignAPAC','DocuSignAPI','DocuSignIMPACT','DocuSignUK','DocuSignING']
signnow = ['signnow']

From this observation, I found that it would be possible to label for spam_corporate based on that whether screen name contains certain company name or not (would be more efficient)

* bonus found: Adobe leverages its branch companies
* Q: Do we need multiple accounts or one accoun for social media marketing?

In [5]:
combined_lst = adobe+onespan+docusign+signnow

In [6]:
#df[df['spam_']]

In [7]:
#df.loc[df['screen_name'] in (combined_lst), 'spam_corporate'] = 1

In [8]:
#df.loc[df['c1'] == 'Value', 'c2'] = 10

In [9]:
df['spam_corporate'][df.screen_name.isin(combined_lst)] = 1

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


2. Bot generated post

Bot generated post for this case is by AdobeSign, in format of "Just closed a deal in..." / "Just signed an agreement in.." / "just connected my...". its expanded url column value is always adobe sign. 

For right now, we can use the assumption that if expanded url column value is link to AdobeSign, then it is bot generated post. However, it would be possible to process text and compare its vector (ex: use clustering) to capture the group of bot generated post with its distinct pattern (later task to elaborate the work)

Considering time restriction, I will proceed with this hypothesis.

In [17]:
df['spam_bot'][df.expanded_url == 'https://acrobat.adobe.com'] = 1

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
df.head()

,index,text,retweet_count,favorite_count,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,...,place_type,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url
0,0,just closed a deal in 27 hours using #AdobeSig...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Fe0YfarG31
1,1,just closed a deal in 2 days using #AdobeSign ...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/TTlCvaDE0V
2,2,just closed a deal in 2 hours using #AdobeSign...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/okmHDeM9bZ
3,3,just closed a deal in 26 hours using #AdobeSig...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/AAhUGDLZxH
4,4,just closed a deal in 6 days using #AdobeSign ...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Oi57q6jdXP


Known spammer, Own Post spam would be labeled with similar method like below:

In [ ]:
x = input("Type in target user name: ")

In [32]:
df.columns

Index(['index', 'text', 'retweet_count', 'favorite_count', 'spam',
       'spam_marketing', 'spam_hijack', 'spam_corporate', 'spam_bot',
       'spam_known', 'spam_own', 'Docusign', 'onespan', 'signnow',
       'adobe sign', 'favorited', 'truncated', 'id_str',
       'in_reply_to_screen_name', 'source', 'retweeted', 'created_at',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'lang',
       'listed_count', 'verified', 'location', 'user_id_str', 'description',
       'geo_enabled', 'user_created_at', 'statuses_count', 'followers_count',
       'favourites_count', 'protected', 'user_url', 'name', 'time_zone',
       'user_lang', 'utc_offset', 'friends_count', 'screen_name',
       'country_code', 'country', 'place_type', 'full_name', 'place_name',
       'place_id', 'place_lat', 'place_lon', 'lat', 'lon', 'expanded_url',
       'url'],
      dtype='object')

And... finally eyeball time.
Let's seperate out the rows that none of spam categories value is 1.

In [34]:
df.loc[df['spam_bot'] == 1 | df['spam_corporate'] == 1]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().